In [ ]:
#!pip install flair

In [8]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 14:38:57--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 122.13K  --.-KB/s    in 0.08s   

2021-02-09 14:38:58 (1.47 MB/s) - ‘train.conll’ saved [125059/125059]

--2021-02-09 14:38:58--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [text/plain]
Saving to: ‘dev.conll’

dev.conll           100%[===============

In [9]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings

In [10]:
import flair
import torch
# uncomment for cpu-only training
#flair.device = torch.device('cpu')

In [11]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-09 14:39:02,475 Reading data from .
2021-02-09 14:39:02,476 Train: train.conll
2021-02-09 14:39:02,477 Dev: dev.conll
2021-02-09 14:39:02,477 Test: test.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [12]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    CharacterEmbeddings()
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>


In [13]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
history = trainer.train(
    './models/baseline-charembeddings',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=20
)

2021-02-09 14:39:09,550 ----------------------------------------------------------------------------------------------------
2021-02-09 14:39:09,550 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=50, out_features=50, bias=True)
  (rnn): LSTM(50, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=14, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-02-09 14:39:09,551 ----------------------------------------------------------------------------------------------------
2021-02-09 14:39:09,551 Corpus: "Corpus: 1966 train + 219 dev + 547 test sentences"
2021-02-09 14:39:09,551 -------------------------------------------------------------------------------